In [1]:
import sys
# sys.path.append('/Users/liangwang/GitHub/DataAnalysis')
sys.path.append(r'C:\Users\lwang\Documents\GitHub\DataAnalysis2023')

In [2]:
import numpy as np
import sys, os
from pathlib import Path
import pandas as pd
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
from sklearn.linear_model import LinearRegression
import scipy
import importlib

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

import GlucoseModelsGen1
import ErrorAndCostFunc

load data

In [3]:
df=pd.read_csv(r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen1 Modeling\biolinq vs adj ref training updated.csv', engine='python')
df_orig=df.copy()

Setting

In [25]:
importlib.reload(GlucoseModelsGen1)
importlib.reload(ErrorAndCostFunc)

<module 'ErrorAndCostFunc' from 'c:\\Users\\lwang\\Documents\\GitHub\\for_test_LW\\ErrorAndCostFunc.py'>

In [26]:
gluc_model_optim = GlucoseModelsGen1.GlucoseModel()

In [27]:
training_group = ['eBlinq19c', 'eBlinq20', 'eBlinq22 - Pre-Piv 1']
test_group = ['eBlinq Pre-Piv 2']
all_training_channel = ['Ch4']
training_time_window = [0.083,5]
training_chemistry = '2.6.11'
model_output_valid_thresh = 0.5
all_model_name = [
    # 'sim_slope',
    # 'sim_slope_constant_background',
    'sim_slope_constant_compartment',
    # 'sim_slope_linear_background',
    # 'sim_slope_nonlinear_background',
    # 'sim_slope_intercept_constant_background',
    # 'sim_slope_intercept_constant_compartment',
    # 'sim_slope_intercept_linear_background',
    # 'sim_slope_intercept_nonlinear_background',
    # 'fmm_constant_background',
    # 'fmm_constant_compartment',
    # 'fmm_linear_background',
    # 'fmm_nonlinear_background'
    ]
all_err_func_name = [
    # 'g_error_mse',
    # 'g_error_mard',
    # 'g_error_mard_weighted',
    # 'g_error_median_ard',
    # 'cur_error_mse',
    # 'cur_error_mard',
    # 'cur_error_mard_weighted',
    'cur_error_median_ard'
]
all_results = {}
all_results['metric'] = [
    'mard_training',
    'mard_weighted_training',
    'mard_test',
    'mard_weighted_test',
    'ls_score_training',
    'ls_slope_training',
    'ls_intercept_training',
    'ls_score_test',
    'ls_slope_test',
    'ls_intercept_test'
]
all_optim_params = {}

In [28]:
for training_channel in all_training_channel:
    for model_name in all_model_name:
        for err_func_name in all_err_func_name:
            # training_channel = 'Ch1'
            # model_name = 'sim_slope_linear_background'
            model_gluc = f'{model_name}_gluc'
            model_cur = f'{model_name}_cur'
            bounds = f'{model_name}_bounds'
            err_func = getattr(ErrorAndCostFunc,err_func_name)
            this_result = []
            df = df_orig.copy()
            valid_training_index = df[df['Grouping'].isin(training_group) & df['time_from_start'].between(training_time_window[0],training_time_window[1]) & 
                                    df['Channel'].eq(training_channel) & df['is_model_output_valid'].ge(model_output_valid_thresh) &
                                    df['Chemistry'].eq(training_chemistry)].index
            # df_training_valid = df.loc[valid_training_index].reset_index(drop=True).replace([np.inf, -np.inf], np.nan).dropna()
            df_training_valid = df.loc[valid_training_index].reset_index(drop=True)
            valid_test_index = df[df['Grouping'].isin(test_group) & df['time_from_start'].between(training_time_window[0],training_time_window[1]) & 
                                    df['Channel'].eq(training_channel) & df['is_model_output_valid'].ge(model_output_valid_thresh) &
                                    df['Chemistry'].eq(training_chemistry)].index
            # df_test_valid = df.loc[valid_test_index].reset_index(drop=True).replace([np.inf, -np.inf], np.nan).dropna()
            df_test_valid = df.loc[valid_test_index].reset_index(drop=True)

            g_train = df_training_valid['Ref_value']
            sim_train = df_training_valid['mag_1007']
            i_train = df_training_valid['current']

            g_test = df_test_valid['Ref_value']
            sim_test = df_test_valid['mag_1007']
            i_test = df_test_valid['current']

            glucose_model = getattr(gluc_model_optim,model_gluc)
            current_model = getattr(gluc_model_optim,model_cur)
            bounds = getattr(gluc_model_optim,bounds)

            if err_func_name.startswith('g'):
                result= scipy.optimize.dual_annealing(err_func,bounds,args=(sim_train,i_train,g_train,glucose_model),maxiter=int(1e4))
            elif err_func_name.startswith('cur'):
                result= scipy.optimize.dual_annealing(err_func,bounds,args=(sim_train,i_train,g_train,current_model),maxiter=int(1e4))

            # result_g_error_mse = scipy.optimize.dual_annealing(ErrorAndCostFunc.g_error_mse,bounds,args=(sim_train,i_train,g_train,glucose_model),maxiter=int(1e4))
            # result_g_error_mard = scipy.optimize.dual_annealing(ErrorAndCostFunc.g_error_mard,bounds,args=(sim_train,i_train,g_train,glucose_model),maxiter=int(1e4))
            # result_g_error_mard_weighted = scipy.optimize.dual_annealing(ErrorAndCostFunc.g_error_mard_weighted,bounds,args=(sim_train,i_train,g_train,glucose_model),maxiter=int(1e4))
            # result_g_error_median_ard = scipy.optimize.dual_annealing(ErrorAndCostFunc.g_error_median_ard,bounds,args=(sim_train,i_train,g_train,glucose_model),maxiter=int(1e4))
            # result_cur_error_mse = scipy.optimize.dual_annealing(ErrorAndCostFunc.cur_error_mse,bounds,args=(sim_train,i_train,g_train,current_model),maxiter=int(1e4))
            # result_cur_error_mard = scipy.optimize.dual_annealing(ErrorAndCostFunc.cur_error_mard,bounds,args=(sim_train,i_train,g_train,current_model),maxiter=int(1e4))
            # result_cur_error_mard_weighted = scipy.optimize.dual_annealing(ErrorAndCostFunc.cur_error_mard_weighted,bounds,args=(sim_train,i_train,g_train,current_model),maxiter=int(1e4))
            # result_cur_error_median_ard = scipy.optimize.dual_annealing(ErrorAndCostFunc.cur_error_median_ard,bounds,args=(sim_train,i_train,g_train,current_model),maxiter=int(1e4))

            # ErrorAndCostFunc.g_error_mse(result.x,sim_train,i_train,g_train,glucose_model)
            # ErrorAndCostFunc.g_error_mse(result.x,sim_test,i_test,g_test,glucose_model)
            this_result.append(ErrorAndCostFunc.g_error_mard(result.x,sim_train,i_train,g_train,glucose_model))
            this_result.append(ErrorAndCostFunc.g_error_mard_weighted(result.x,sim_train,i_train,g_train,glucose_model))
            this_result.append(ErrorAndCostFunc.g_error_mard(result.x,sim_test,i_test,g_test,glucose_model))
            this_result.append(ErrorAndCostFunc.g_error_mard_weighted(result.x,sim_test,i_test,g_test,glucose_model))

            reg_train = LinearRegression().fit(g_train.values.reshape(-1,1), glucose_model(sim_train,i_train,result.x).values)
            this_result.append(reg_train.score(g_train.values.reshape(-1,1), glucose_model(sim_train,i_train,result.x).values))
            this_result.append(reg_train.coef_)
            this_result.append(reg_train.intercept_)           
            # print(reg_train.score(g_train.values.reshape(-1,1), glucose_model(sim_train,i_train,result.x).values))
            # print(reg_train.coef_)
            # print(reg_train.intercept_)

            reg_test = LinearRegression().fit(g_test.values.reshape(-1,1), glucose_model(sim_test,i_test,result.x).values)
            this_result.append(reg_test.score(g_test.values.reshape(-1,1), glucose_model(sim_test,i_test,result.x).values))
            this_result.append(reg_test.coef_)
            this_result.append(reg_test.intercept_)
            # print(reg_test.score(g_test.values.reshape(-1,1), glucose_model(sim_test,i_test,result.x).values))
            # print(reg_test.coef_)
            # print(reg_test.intercept_)

            all_optim_params[f'{model_name}_{err_func_name}_{training_channel}'] = result.x
            all_results[f'{model_name}_{err_func_name}_{training_channel}'] = this_result
            results_df = pd.DataFrame.from_dict(all_results)
            results_df.to_csv(r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen1 Modeling\results_13_models_ch4_candidate3.csv',header=True)

In [31]:
results_df

,metric,sim_slope_constant_compartment_cur_error_median_ard_Ch4
0,mard_training,25.892797
1,mard_weighted_training,31.362246
2,mard_test,24.869638
3,mard_weighted_test,29.328969
4,ls_score_training,0.767686
5,ls_slope_training,[1.0159823047199292]
6,ls_intercept_training,2.290211
7,ls_score_test,0.765832
8,ls_slope_test,[0.9805961852152751]
9,ls_intercept_test,4.911256


In [29]:
all_optim_params

{'sim_slope_constant_compartment_cur_error_median_ard_Ch4': array([843.95453245, -22.37337731])}

In [30]:
1/all_optim_params['sim_slope_g_error_mard_weighted_Ch4']

KeyError: 'sim_slope_g_error_mard_weighted_Ch4'

In [ ]:
# #quadruple low glucose
# df_training_valid=df_training_valid.append(df_training_valid[df_training_valid['Ref']<80]).reset_index(drop=True)
# df_training_valid=df_training_valid.append(df_training_valid[df_training_valid['Ref']<80]).reset_index(drop=True)

In [ ]:
bounds

[[0.001, 2000.0]]

In [ ]:
result

 message: ['Maximum number of iteration reached']
 success: True
  status: 0
     fun: 28.345808730150658
       x: [ 9.794e+02]
     nit: 10000
    nfev: 74211
    njev: 27101
    nhev: 0

In [ ]:
sim_train.shape

(73852,)

In [ ]:
sim_test.shape

(69121,)